In [7]:
import numpy as np 

def check_advantage(dice_sides):
    if dice_sides[-1] == 'a':
        return 1
    if dice_sides[-1] == 'd':
        return -1
    else:
        return 0

def split_input(string):
    return string.split(',')

def comment_criticals(message, fails, hits):
    if fails == 1:
        message += f'. 1 CRITICAL FAILURE'
    if hits == 1:
        message += f'. 1 CRITICAL SUCCESS'
    if fails > 1:
        message += f'. {fails} CRITICAL FAILURES'
    if hits > 1:
        message += f'. {hits} CRITICAL SUCCESSES!'
    return message

def check_if_modifier(dice_input):
    first_character = dice_input.strip()[0]
    if first_character == '+':
        return 1
    if first_character == '-':
        return -1
    if first_character != '+' and first_character != '-':
        return 0

def check_if_dice_modifier(roll_list):
    for i in range(0, len(roll_list)):
        if i != len(roll_list)-1:
            if roll_list[i+1]['is_modifier'] == 1 or roll_list[i+1]['is_modifier'] == -1:
                roll_list[i]['has_dice_modifier'] = True
            else:
                roll_list[i]['has_dice_modifier'] = False
    else:
         roll_list[i]['has_dice_modifier'] = False

def split_roll(input):
    split_1 = input.split('d', 1)
    try:
        dice_count = int(split_1[0].strip())
    except:
        dice_count = 1
    if '+' not in split_1[1] and '-' not in split_1[1]:
        dice_sides = split_1[1].strip()
        advantage = check_advantage(dice_sides)
        if advantage == 0:
            dice_sides = int(dice_sides)
        else:
            dice_sides = int(dice_sides[0:-1])
        modifier = 0
        return dice_count, dice_sides, advantage, modifier
    if '+' in split_1[1]:
        split_2 = split_1[1].split('+')
        modifier = int(split_2[1].strip())
        dice_sides = split_2[0].strip()
        advantage = check_advantage(dice_sides)
        if advantage == 0:
            dice_sides = int(dice_sides)
        else:
            dice_sides = int(dice_sides[0:-1])
        return dice_count, dice_sides, advantage, modifier
    if '-' in split_1[1]:
        split_2 = split_1[1].split('-')
        modifier = 0 - int(split_2[1].strip())
        dice_sides = split_2[0].strip()
        advantage = check_advantage(dice_sides)
        if advantage == 0:
            dice_sides = int(dice_sides)
        else:
            dice_sides = int(dice_sides[0:-1])
        return dice_count, dice_sides, advantage, modifier

def count_criticals(rolls):
    critical_fails = np.count_nonzero(1 == rolls)
    critical_hits = np.count_nonzero(20 == rolls)
    return critical_fails, critical_hits

def roll_dice(dice_count, dice_sides, advantage):
    if advantage == 1 or advantage == -1:
        rolls = []
        for i in range(0, dice_count):
            rolls.append(np.random.randint(1, dice_sides+1, 2))
        fails, hits = 0, 0
        if dice_sides == 20:
            fails, hits = count_criticals(rolls)
        return rolls, fails, hits
    else:
        rolls = np.random.randint(1, dice_sides+1, dice_count)
        fails, hits = 0, 0
        if dice_sides == 20:
            fails, hits = count_criticals(rolls)
        return rolls, fails, hits



def roll_them_dice(user_input_string):
    to_roll = split_input(user_input_string)
    roll_list = []
    roll_number = 0

#get the info for each dice
    for roll in to_roll:
        roll_dict = {}
        roll_dict['is_modifier'] = check_if_modifier(roll)
        if roll_dict['is_modifier'] == 0:
            roll_dict['dice_count'], roll_dict['dice_sides'], roll_dict['advantage'], roll_dict['modifier'] = split_roll(roll)
            roll_dict['roll_number'] = roll_number
            roll_list.append(roll_dict)
            roll_number += 1
        else: 
            roll_dict['dice_count'], roll_dict['dice_sides'], roll_dict['advantage'], roll_dict['modifier'] = split_roll(roll.strip()[1:])
            roll_dict['roll_number'] = roll_number
            roll_list.append(roll_dict)
            roll_number += 1

    for roll in roll_list:
        roll['result'], roll['fails'], roll['hits'] = roll_dice(roll['dice_count'], roll['dice_sides'], roll['advantage'])

    check_if_dice_modifier(roll_list)

    roll_number = 0
# having collected the info for each dice, we now turn these into outputs 

    for roll in roll_list:
# for very simple rolls of a single dice with no modifiers or advantage
        if roll['dice_count'] == 1 and roll['is_modifier'] == 0 and roll['modifier'] == 0 and roll['has_dice_modifier'] == 0 and roll['advantage'] == 0:
            result_string = str((roll['result']))[1:-1]
            message = f' rolled **{result_string}**.'
            roll['message'] = message

# for rolls (like damage rolls) that are not d20 and don't have advantage
        elif roll['dice_sides'] != 20 and roll['is_modifier'] == 0 and roll['advantage'] == 0:
            try:
                if roll['has_dice_modifier']:
                    modifier_2 = roll_list[roll_number+1]['result'].sum() + roll_list[roll_number+1]['modifier']
                    if roll_list[roll_number+1]['is_modifier'] == -1:
                        modifier_2 = -1 * modifier_2
                else:
                    modifier_2 = 0
            except:
                modifier_2 = 0
            total_modifier = roll['modifier'] + modifier_2
            final_roll = roll['result'].sum() + total_modifier
            sum_statement = ''
            modifier_string = ''
            if roll['modifier'] != 0:
                modifier_string = ' + ' + str(roll['modifier'])
            if modifier_2 != 0:
                modifier_string += f' + {str(modifier_2)}'
            for i in range(0, len(roll['result'])):
                if i != len(roll['result'])-1:
                    roll_string = str(roll['result'][i])
                    sum_statement += f'{roll_string} + '
                else:
                    sum_statement += str(roll['result'][i])
            message = f' rolled **{final_roll}**. (({sum_statement}){modifier_string} = {final_roll})'
            fails, hits = count_criticals(roll['result'])
            roll['message'] = message

# for rolls with advantage we will want to see all the rolls + modifiers
        elif roll['dice_sides'] == 20 and roll['advantage'] != 0 and roll['is_modifier'] == 0:
            try:
                if roll['has_dice_modifier']:
                    modifier_2 = roll_list[roll_number+1]['result'].sum() + roll_list[roll_number+1]['modifier']
                    if roll_list[roll_number+1]['is_modifier'] == -1:
                        modifier_2 = -1 * modifier_2
                else:
                    modifier_2 = 0
            except:
                modifier_2 = 0
            total_modifier = roll['modifier'] + modifier_2
            if roll['advantage'] == 1:
                outcomes = [x.max() for x in roll['result']]
            if roll['advantage'] == -1:
                outcomes = [x.min() for x in roll['result']]
            final_outcomes = np.array(outcomes) + total_modifier
            final_outcomes_statement = str(list(final_outcomes))[1:-1].strip().replace('  ', ' ').replace(' ', ' & ')
            final_outcomes_formatted = final_outcomes_statement.replace(' & ', '** & **').replace(',', ' ')
            # make a string for modifiers
            modifier_string = ''
            if roll['modifier'] != 0:
                modifier_string = ' + ' + str(roll['modifier'])
            if modifier_2 != 0:
                modifier_string += f' + {str(modifier_2)}'
            sum_statement = ''
            for i in range(0, len(roll['result'])):
                roll_tuple = roll['result'][i]
                if roll['advantage'] == 1:
                    if i != len(roll['result'])-1:
                        sum_statement += f'({roll_tuple.min()} , **{roll_tuple.max()}**){modifier_string} = {final_outcomes[i]}, '
                    else:
                        sum_statement += f'({roll_tuple.min()} , **{roll_tuple.max()}**){modifier_string} = {final_outcomes[i]}'
                if roll['advantage'] == -1:
                    if i != len(roll['result'])-1:
                        sum_statement += f'(**{roll_tuple.min()}** , {roll_tuple.max()}){modifier_string} = {final_outcomes[i]}, '
                    else:
                        sum_statement += f'(**{roll_tuple.min()}** , {roll_tuple.max()}){modifier_string} = {final_outcomes[i]}'
            message = f' rolled **{final_outcomes_formatted}**. ({sum_statement})'
            fails, hits = count_criticals(roll['result'])
            message = comment_criticals(message, fails, hits)
            roll['message'] = message

# for rolls of d20 that do not have advantage, where we want to see each roll + modifiers
        elif roll['dice_sides'] == 20 and roll['advantage'] == 0 and roll['is_modifier'] == 0:
            try:
                if roll['has_dice_modifier']:
                    modifier_2 = roll_list[roll_number+1]['result'].sum() + roll_list[roll_number+1]['modifier']
                    if roll_list[roll_number+1]['is_modifier'] == -1:
                        modifier_2 = -1 * modifier_2
                else:
                    modifier_2 = 0
            except:
                modifier_2 = 0
            total_modifier = roll['modifier'] + modifier_2
            final_outcomes = np.array(roll['result']) + total_modifier
            final_outcomes_statement = str(final_outcomes)[1:-1].strip().replace('  ', ' ').replace(' ', ' & ')
            final_outcomes_formatted = final_outcomes_statement.replace(' & ', '** & **')
            modifier_string = ''
            if roll['modifier'] != 0:
                modifier_string = ' + ' + str(roll['modifier'])
            if modifier_2 != 0:
                modifier_string += f' + {str(modifier_2)}'
            sum_statement = ''
            if modifier_string != '':
                for i in range(0, len(roll['result'])):
                    result_str = str(roll['result'][i])
                    if i != len(roll['result'])-1:
                        sum_statement += f'({result_str}){modifier_string} = {final_outcomes[i]}, '
                    else:
                        sum_statement += f'({result_str}){modifier_string} = {final_outcomes[i]}'
                sum_statement = f'({sum_statement})'
            message = f' rolled **{final_outcomes_formatted}**. {sum_statement}'
            fails, hits = count_criticals(roll['result'])
            message = comment_criticals(message, fails, hits)
            roll['message'] = message
    roll_number += 1

    return roll_list





In [175]:
here = '1d20+5, +d6, +d5'

In [178]:
rolls = roll_them_dice(here)
for roll in rolls:
    try:
        print(roll['message'])
    except:
        pass

 rolled **17**. ((9) + 8 = 17)
 rolled **10**. ((8 + 1) + 1 = 10)
 rolled **11**. ((7 + 4) = 11)
 rolled **12** & **20** & **8**. ((8) + 4 = 12, (16) + 4 = 20, (4) + 4 = 8)
 rolled **17 ** & **21**. ((2 , **16**) + 1 = 17, (4 , **20**) + 1 = 21)
 rolled **22 ** & **15 ** & **25**. ((9 , **13**) + 1 + 8 = 22, (2 , **6**) + 1 + 8 = 15, (4 , **16**) + 1 + 8 = 25)
 rolled **19** & **18**. 
 rolled **3**.


In [177]:
here = 'd20, + 2d6, 2d8+1, 2d8, 3d20+4, 2d20a+1, 3d20a+1, +d3, 2d20, 1d10'

In [156]:
rolls

[{'is_modifier': 0,
  'dice_count': 1,
  'dice_sides': 20,
  'advantage': 0,
  'modifier': 5,
  'roll_number': 0,
  'result': array([2]),
  'fails': 0,
  'hits': 0,
  'has_dice_modifier': True,
  'message': ' rolled **10**. '},
 {'is_modifier': 1,
  'dice_count': 1,
  'dice_sides': 6,
  'advantage': 0,
  'modifier': 0,
  'roll_number': 1,
  'result': array([3]),
  'fails': 0,
  'hits': 0,
  'has_dice_modifier': False}]

In [5]:
message = '~roll d20'
message[0:5], message[5:]

('~roll', ' d20')

In [8]:
roll_them_dice(message[5:])

[{'is_modifier': 0,
  'dice_count': 1,
  'dice_sides': 20,
  'advantage': 0,
  'modifier': 0,
  'roll_number': 0,
  'result': array([14]),
  'fails': 0,
  'hits': 0,
  'has_dice_modifier': False,
  'message': ' rolled **14**.'}]

In [130]:
mess = '!r bird'
if mess[0:3] == '!r ':
    print(mess)

!r bird


In [2]:
dice_count, dice_sides, advantage, modifier = split_roll('4d20a+4')
result, fails, hits = roll_dice(dice_count, dice_sides, advantage)

build_message(result, modifier, fails, hits, advantage)

' rolled **55**. ((5 , **20**) + (6 , **11**) + (4 , **6**) + (3 , **14**)) + 4 = 55)'

### outline:

1. split the input into different dice rolls

2. for each dice roll, tell if it is to be:
- added to first
- subtracted from first
- rolled as an independent additional roll

3. for each roll, generate:
- number of dice
- sides of dice
- modifiers
- whether it has advantage, disadvantage, or neither

4. roll each dice and store the values

5. 

In [105]:
def roll_them_dice(user_input_string):
    to_roll = split_input(user_input_string)
    roll_list = []
    roll_number = 0

    for roll in split_input(example):
        roll_dict = {}
        roll_dict['is_modifier'] = check_if_modifier(roll)
        if roll_dict['is_modifier'] == 0:
            roll_dict['dice_count'], roll_dict['dice_sides'], roll_dict['advantage'], roll_dict['modifier'] = split_roll(roll)
            roll_dict['roll_number'] = roll_number
            roll_list.append(roll_dict)
            roll_number += 1
        else: 
            roll_dict['dice_count'], roll_dict['dice_sides'], roll_dict['advantage'], roll_dict['modifier'] = split_roll(roll.strip()[1:])
            roll_dict['roll_number'] = roll_number
            roll_list.append(roll_dict)
            roll_number += 1

    for roll in roll_list:
        roll['result'], roll['fails'], roll['hits'] = roll_dice(roll['dice_count'], roll['dice_sides'], roll['advantage'])

    check_if_dice_modifier(roll_list)

    roll_number = 0
# having collected the info for each dice, we now turn these into outputs 

    for roll in roll_list:
# for very simple rolls of a single dice with no modifiers or advantage
        if roll['dice_count'] == 1 and roll['is_modifier'] == 0 and roll['modifier'] == 0:
            result_string = str(roll['result'])[1:-1]
            message = f' rolled **{result_string}**.'
            roll['message'] = message

# for rolls (like damage rolls) that are not d20 and don't have advantage
        elif roll['dice_sides'] != 20 and roll['is_modifier'] == 0 and roll['advantage'] == 0:
            try:
                if roll['has_dice_modifier']:
                    modifier_2 = roll_list[roll_number+1]['result'].sum() + roll_list[roll_number+1]['modifier']
                    if roll_list[roll_number+1]['is_modifier'] == -1:
                        modifier_2 = -1 * modifier_2
                else:
                    modifier_2 = 0
            except:
                modifier_2 = 0
            total_modifier = roll['modifier'] + modifier_2
            final_roll = roll['result'].sum() + total_modifier
            sum_statement = ''
            if roll['modifier'] != 0:
                modifier_string = ' + ' + str(roll['modifier'])
            if modifier_2 != 0:
                modifier_string += f' + {str(modifier_2)}'
            for i in range(0, len(roll['result'])):
                if i != len(roll['result'])-1:
                    roll_string = str(roll['result'][i])
                    sum_statement += f'{roll_string} + '
                else:
                    sum_statement += str(roll['result'][i])
            modifier_string = str(roll['modifier'])
            if modifier_2 == 0:
                message = f' rolled **{final_roll}**. (({sum_statement}) + {modifier_string} = {final_roll})'
            fails, hits = count_criticals(roll['result'])
            roll['message'] = message

# for rolls with advantage we will want to see all the rolls + modifiers
        elif roll['dice_sides'] == 20 and roll['advantage'] != 0 and roll['is_modifier'] == 0:
            try:
                if roll['has_dice_modifier']:
                    modifier_2 = roll_list[roll_number+1]['result'].sum() + roll_list[roll_number+1]['modifier']
                    if roll_list[roll_number+1]['is_modifier'] == -1:
                        modifier_2 = -1 * modifier_2
                else:
                    modifier_2 = 0
            except:
                modifier_2 = 0
            total_modifier = roll['modifier'] + modifier_2
            if roll['advantage'] == 1:
                outcomes = [x.max() for x in roll['result']]
            if roll['advantage'] == -1:
                outcomes = [x.min() for x in roll['result']]
            final_outcomes = np.array(outcomes) + total_modifier
            final_outcomes_statement = str(final_outcomes)[1:-1].strip().replace('  ', ' ').replace(' ', ' & ')
            final_outcomes_formatted = final_outcomes_statement.replace(' & ', '** & **')
            # make a string for modifiers
            if roll['modifier'] != 0:
                modifier_string = ' + ' + str(roll['modifier'])
            if modifier_2 != 0:
                modifier_string += f' + {str(modifier_2)}'
            sum_statement = ''
            for i in range(0, len(roll['result'])):
                roll_tuple = roll['result'][i]
                if roll['advantage'] == 1:
                    if i != len(roll['result'])-1:
                        sum_statement += f'({roll_tuple.min()} , **{roll_tuple.max()}**){modifier_string} = {final_outcomes[i]}, '
                    else:
                        sum_statement += f'({roll_tuple.min()} , **{roll_tuple.max()}**){modifier_string} = {final_outcomes[i]}'
                if roll['advantage'] == -1:
                    if i != len(roll['result'])-1:
                        sum_statement += f'(**{roll_tuple.min()}** , {roll_tuple.max()}){modifier_string} = {final_outcomes[i]}, '
                    else:
                        sum_statement += f'(**{roll_tuple.min()}** , {roll_tuple.max()}){modifier_string} = {final_outcomes[i]}'
            message = f' rolled **{final_outcomes_formatted}**. ({sum_statement})'
            fails, hits = count_criticals(roll['result'])
            message = comment_criticals(message, fails, hits)
            roll['message'] = message

# for rolls of d20 that do not have advantage, where we want to see each roll + modifiers
        elif roll['dice_sides'] == 20 and roll['advantage'] == 0 and roll['is_modifier'] == 0:
            try:
                if roll['has_dice_modifier']:
                    modifier_2 = roll_list[roll_number+1]['result'].sum() + roll_list[roll_number+1]['modifier']
                    if roll_list[roll_number+1]['is_modifier'] == -1:
                        modifier_2 = -1 * modifier_2
                else:
                    modifier_2 = 0
            except:
                modifier_2 = 0
            total_modifier = roll['modifier'] + modifier_2
            final_outcomes = np.array(roll['result']) + total_modifier
            final_outcomes_statement = str(final_outcomes)[1:-1].strip().replace('  ', ' ').replace(' ', ' & ')
            final_outcomes_formatted = final_outcomes_statement.replace(' & ', '** & **')
            if roll['modifier'] != 0:
                modifier_string = ' + ' + str(roll['modifier'])
            if modifier_2 != 0:
                modifier_string += f' + {str(modifier_2)}'
            sum_statement = ''
            for i in range(0, len(roll['result'])):
                result_str = str(roll['result'][i])
                if i != len(roll['result'])-1:
                    sum_statement += f'({result_str}){modifier_string} = {final_outcomes[i]}, '
                else:
                    sum_statement += f'({result_str}){modifier_string} = {final_outcomes[i]}'
            message = f' rolled **{final_outcomes_formatted}**. ({sum_statement})'
            fails, hits = count_criticals(roll['result'])
            message = comment_criticals(message, fails, hits)
            roll['message'] = message
    roll_number += 1

    return roll_list





In [106]:
example = '4d20a+9, - 2d4, 2d6+5, 3d20d + 1, 3d6+15, 1d6'

In [140]:
roll_them_dice(example)

[{'is_modifier': 0,
  'dice_count': 4,
  'dice_sides': 20,
  'advantage': 1,
  'modifier': 9,
  'roll_number': 0,
  'result': [array([ 7, 16]), array([6, 1]), array([ 9, 12]), array([13, 13])],
  'fails': 0,
  'hits': 0,
  'has_dice_modifier': True,
  'message': ' rolled **18 & 8 & 14 & 15**. ((7 , **16**) + 9 + -7 = 18, (1 , **6**) + 9 + -7 = 8, (9 , **12**) + 9 + -7 = 14, (13 , **13**) + 9 + -7 = 15)'},
 {'is_modifier': -1,
  'dice_count': 2,
  'dice_sides': 4,
  'advantage': 0,
  'modifier': 0,
  'roll_number': 1,
  'result': array([4, 3]),
  'fails': 0,
  'hits': 0,
  'has_dice_modifier': False},
 {'is_modifier': 0,
  'dice_count': 2,
  'dice_sides': 6,
  'advantage': 0,
  'modifier': 5,
  'roll_number': 2,
  'result': array([6, 6]),
  'fails': 0,
  'hits': 0,
  'has_dice_modifier': False,
  'message': ' rolled **17**. ((6 + 6) + 5 = 17)'},
 {'is_modifier': 0,
  'dice_count': 3,
  'dice_sides': 20,
  'advantage': -1,
  'modifier': 1,
  'roll_number': 3,
  'result': [array([ 5, 12])

In [90]:
split_input(example)

['4d20a+9', ' + 2d4', ' 2d6+5', ' 3d20 + 1', ' 3d6+15']

In [182]:
roll_list = []
roll_number = 0

for roll in split_input(example):
    roll_dict = {}
    roll_dict['is_modifier'] = check_if_modifier(roll)
    if roll_dict['is_modifier'] == 0:
        roll_dict['dice_count'], roll_dict['dice_sides'], roll_dict['advantage'], roll_dict['modifier'] = split_roll(roll)
        roll_dict['roll_number'] = roll_number
        roll_list.append(roll_dict)
        roll_number += 1
    else: 
        roll_dict['dice_count'], roll_dict['dice_sides'], roll_dict['advantage'], roll_dict['modifier'] = split_roll(roll.strip()[1:])
        roll_dict['roll_number'] = roll_number
        roll_list.append(roll_dict)
        roll_number += 1

roll_list

[{'is_modifier': 0,
  'dice_count': 2,
  'dice_sides': 20,
  'advantage': 1,
  'modifier': 4,
  'roll_number': 0},
 {'is_modifier': 1,
  'dice_count': 2,
  'dice_sides': 4,
  'advantage': 0,
  'modifier': 0,
  'roll_number': 1},
 {'is_modifier': 0,
  'dice_count': 2,
  'dice_sides': 6,
  'advantage': 0,
  'modifier': 5,
  'roll_number': 2},
 {'is_modifier': 0,
  'dice_count': 1,
  'dice_sides': 6,
  'advantage': 0,
  'modifier': 0,
  'roll_number': 3}]

In [132]:
for roll in roll_list:
    roll['result'], roll['fails'], roll['hits'] = roll_dice(roll['dice_count'], roll['dice_sides'], roll['advantage'])

check_if_dice_modifier(roll_list)
roll_list

NameError: name 'roll_list' is not defined

In [104]:
statement = '15 & 6 & 9'
statement.replace(' & ', '** & **')

'15** & **6** & **9'

to build a message

first check if it was rolled with advantage
if so, you will want the message to display a bunch of different scores, (the outcome of each roll + the modifier)

if not, you will want to display a single sum for all the dice (eg as in a damage roll)

In [171]:
roll_list[0]['result'][0].min(), roll_list[0]['result'][0].max()

(3, 7)

In [172]:
[x.max() for x in roll_list[0]['result']]

[7, 15]

In [170]:
bird = [np.array([5, 3]), np.array([18, 14])]
[x.max() for x in bird]

TypeError: Cannot interpret '3' as a data type

In [28]:
str(final_outcomes)[1:-1].replace(' ', ', ')

'30, 32'

In [ ]:
def build_message(rolls, modifier, fails, hits, advantage):
    if advantage == True:
        maximums = [x.max() for x in rolls]
        final_roll = np.array(maximums).sum() + modifier
        sum_statement = ''
        for i in range(0, len(rolls)):
            if i != len(rolls)-1:
                sum_statement += f'({rolls[i].min()} , **{rolls[i].max()}**) + '
            else:
                sum_statement += f'({rolls[i].min()} , **{rolls[i].max()}**)'
        message = f' rolled **{final_roll}**. ({sum_statement}) + {modifier} = {final_roll})'
    else:    
        final_roll = rolls.sum() + modifier
        sum_statement = ''
        for i in range(0, len(rolls)):
            if i != len(rolls)-1:
                sum_statement += f'{rolls[i]} + '
            else:
                sum_statement += str(rolls[i])
        message = f' rolled **{final_roll}**. (({sum_statement}) + {modifier} = {final_roll})'
    if fails == 1:
        message += f'. 1 CRITICAL FAILURE'
    if hits == 1:
        message += f'. 1 CRITICAL SUCCESS'
    if fails > 1:
        message += f'. {fails} CRITICAL FAILURES'
    if hits > 1:
        message += f'. {hits} CRITICAL SUCCESSES!'
    return message

In [ ]:
def build_message(rolls, modifier, fails, hits, advantage):
    if advantage == True:
        maximums = [x.max() for x in rolls]
        final_roll = np.array(maximums).sum() + modifier
        sum_statement = ''
        for i in range(0, len(rolls)):
            if i != len(rolls)-1:
                sum_statement += f'({rolls[i].min()} , **{rolls[i].max()}**) + '
            else:
                sum_statement += f'({rolls[i].min()} , **{rolls[i].max()}**)'
        message = f' rolled **{final_roll}**. ({sum_statement}) + {modifier} = {final_roll})'
    else:    
        final_roll = rolls.sum() + modifier
        sum_statement = ''
        for i in range(0, len(rolls)):
            if i != len(rolls)-1:
                sum_statement += f'{rolls[i]} + '
            else:
                sum_statement += str(rolls[i])
        message = f' rolled **{final_roll}**. (({sum_statement}) + {modifier} = {final_roll})'
    return message

In [12]:
def check_if_dice_modifier(roll_list):
    for i in range(0, len(roll_list)):
        if i != len(roll_list)-1:
            if roll_list[i+1]['is_modifier'] == 1 or roll_list[i+1]['is_modifier'] == -1:
                roll_list[i]['has_dice_modifier'] = True
            else:
                roll_list[i]['has_dice_modifier'] = False
    else:
         roll_list[i]['has_dice_modifier'] = False


In [61]:
len(roll_list)-1

2

In [63]:
roll_list[0+1]

{'is_modifier': 1,
 'dice_count': 2,
 'dice_sides': 4,
 'advantage': 0,
 'modifier': 0,
 'roll_number': 1,
 'result': array([4, 3]),
 'fails': 0,
 'hits': 0}

In [8]:
message = '!r 2d6'
shave = message[2:]
roll_them_dice(shave)

[{'is_modifier': 0,
  'dice_count': 2,
  'dice_sides': 6,
  'advantage': 0,
  'modifier': 0,
  'roll_number': 0,
  'result': array([5, 6]),
  'fails': 0,
  'hits': 0,
  'has_dice_modifier': False,
  'message': ' rolled **11**. ((5 + 6) + 0 = 11)'}]

In [202]:
'd342'.split('d')

['', '342']

In [6]:
command = '!d 2d5'
command[0:3], command[3:]

('!d ', '2d5')

In [ ]:
if modifier_2 